# 单卡GPU 进行 ChatGLM3-6B模型 LORA 高效微调
本 Cookbook 将带领开发者使用 `AdvertiseGen` 对 ChatGLM3-6B 数据集进行 lora微调，使其具备专业的广告生成能力。

## 硬件需求
显存：24GB
显卡架构：安培架构（推荐）
内存：16GB

## 1. 准备数据集
我们使用 AdvertiseGen 数据集来进行微调。从 [Google Drive](https://drive.google.com/file/d/13_vf0xRTQsyneRKdD1bZIr93vBGOczrk/view?usp=sharing) 或者 [Tsinghua Cloud](https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1) 下载处理好的 AdvertiseGen 数据集，将解压后的 AdvertiseGen 目录放到本目录的 `/data/` 下, 例如。
> /media/zr/Data/Code/ChatGLM3/finetune_demo/data/AdvertiseGen

接着，运行本代码来切割数据集

In [1]:
import json
from typing import Union
from pathlib import Path


def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def _mkdir(dir_name: Union[str, Path]):
    dir_name = _resolve_path(dir_name)
    if not dir_name.is_dir():
        dir_name.mkdir(parents=True, exist_ok=False)


def convert_adgen(data_dir: Union[str, Path], save_dir: Union[str, Path]):
    def _convert(in_file: Path, out_file: Path):
        _mkdir(out_file.parent)
        with open(in_file, encoding='utf-8') as fin:
            with open(out_file, 'wt', encoding='utf-8') as fout:
                for line in fin:
                    dct = json.loads(line)
                    sample = {'conversations': [{'role': 'user', 'content': dct['content']},
                                                {'role': 'assistant', 'content': dct['summary']}]}
                    fout.write(json.dumps(sample, ensure_ascii=False) + '\n')

    data_dir = _resolve_path(data_dir)
    save_dir = _resolve_path(save_dir)

    train_file = data_dir / 'train.json'
    if train_file.is_file():
        out_file = save_dir / train_file.relative_to(data_dir)
        _convert(train_file, out_file)

    dev_file = data_dir / 'dev.json'
    if dev_file.is_file():
        out_file = save_dir / dev_file.relative_to(data_dir)
        _convert(dev_file, out_file)


convert_adgen('data/AdvertiseGen', 'data/AdvertiseGen_fix')

## 2. 使用命令行开始微调,我们使用 lora 进行微调
接着，我们仅需要将配置好的参数以命令行的形式传参给程序，就可以使用命令行进行高效微调，这里将 `/media/zr/Data/Code/ChatGLM3/venv/bin/python3` 换成你的 python3 的绝对路径以保证正常运行。

In [1]:
!cat configs/lora.yaml

data_config:
  train_file: train.json
  val_file: dev.json
  test_file: dev.json
  num_proc: 24
max_input_length: 128
max_output_length: 256
training_args:
  # see `transformers.Seq2SeqTrainingArguments`
  output_dir: ./output
  max_steps: 3000
  # settings for data loading
  per_device_train_batch_size: 4
  dataloader_num_workers: 24
  remove_unused_columns: false
  # settings for saving checkpoints
  save_strategy: steps
  save_steps: 1000
  # settings for logging
  log_level: info
  logging_strategy: steps
  logging_steps: 100
  # settings for evaluation
  per_device_eval_batch_size: 16
  evaluation_strategy: steps
  eval_steps: 1500
  # settings for optimizer
  # adam_epsilon: 1e-6
  # uncomment the following line to detect nan or inf values
  # debug: underflow_overflow
  predict_with_generate: true
  # see `transformers.GenerationConfig`
  generation_config:
    max_new_tokens: 256
  # set your absolute deepspeed path here
  #deepspeed: ds_zero_2.json
  # set to true if train wit

In [2]:
!export python_dir='/home/yuxiang/miniconda3/envs/py311_llm_dev/bin' \
&& export base_model_dir='/home/yuxiang/local/ai_workspace/.cache/huggingface/hub/models--THUDM--chatglm3-6b/snapshots/f30825950ce00cb0577bf6a15e0d95de58e328dc' \
&& export CUDA_VISIBLE_DEVICES=0 \
&& ${python_dir}/python3 finetune_hf.py  data/AdvertiseGen_fix  ${base_model_dir}  configs/lora.yaml

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:02<00:00,  3.21it/s]
trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.031217444255383614
--> Model

--> model has 1.949696M params

train_dataset: Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 114599
})
val_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
test_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
--> Sanity check
           '[gMASK]': 64790 -> -100
               'sop': 64792 -> -100
          '<|user|>': 64795 -> -100
                  '': 30910 -> -100
                '\n': 13 -> -100
                  '': 30910 -> -100
                '类型': 33467 -> -100
                 '#': 31010 -> -100
                 '裤': 56532 -> -100
                 '*': 30998 -> -100
                 '版': 55090 -> -100
                 '型': 54888 -> -100
                 '#': 31010 -> -100
                '宽松': 40833 -> -100


## 3. 使用微调的数据集进行推理
在完成微调任务之后，我们可以查看到 `output` 文件夹下多了很多个`checkpoint-*`的文件夹，这些文件夹代表了训练的轮数。
我们选择最后一轮的微调权重，并使用inference进行导入。

In [3]:
!ls output/

checkpoint-1000  checkpoint-2000  checkpoint-3000


In [4]:
!export python_dir='/home/yuxiang/miniconda3/envs/py311_llm_dev/bin' \
&& export CUDA_VISIBLE_DEVICES=0 \
&& ${python_dir}/python3  inference_hf.py output/checkpoint-3000/ --prompt "类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则"

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:03<00:00,  2.31it/s]
这款连衣裙是经典的一字肩款，搭配上透视网纱，性感又浪漫，而木耳边和压褶的设计，又显得格外的别致。而肩部与袖子的拼接设计，更是增加了整件裙子的层次感，显得格外的灵动。而肩部的拉链设计，更是增加了整件裙子的时尚感，搭配上不规则的裙摆，更是增加了整件裙子的灵动感，穿起来非常的显瘦。


## 4. 总结
到此位置，我们就完成了使用单张 GPU Lora 来微调 ChatGLM3-6B 模型，使其能生产出更好的广告。
在本章节中，你将会学会：
+ 如何使用模型进行 Lora 微调
+ 微调数据集的准备和对齐
+ 使用微调的模型进行推理